# 50 Generate Priority Products

Stage: `05_products`
Discipline: decision-support product generation.

Input:
- `outputs/index_pipeline/30_scoring/municipio_indices_scored.csv`

Outputs:
- `outputs/index_pipeline/50_products/priority_actions.csv`
- `outputs/index_pipeline/50_products/index_priority_overview.html`


In [ ]:
# Cell 1: Setup
from pathlib import Path
import pandas as pd


def find_repo_root():
    p = Path.cwd().resolve()
    for c in [p, *p.parents]:
        if (c / "JupyterNotebooks").exists():
            return c
    return p


REPO_ROOT = find_repo_root()
BASE_OUT = REPO_ROOT / "JupyterNotebooks" / "outputs" / "index_pipeline"
INPUT_FILE = BASE_OUT / "30_scoring" / "municipio_indices_scored.csv"
OUTPUT_DIR = BASE_OUT / "50_products"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

if not INPUT_FILE.exists():
    raise FileNotFoundError(f"Missing dependency: {INPUT_FILE}")

idx_df = pd.read_csv(INPUT_FILE)

try:
    from IPython.display import display
except ImportError:
    display = print


In [ ]:
# Cell 2: Build action table and HTML product

def recommend_actions(row):
    actions = []
    if row["flood_hazard_muni"] >= 80:
        actions.append("Pre-position swift-water rescue assets")
        actions.append("Issue targeted flood risk communications")
    if row["earthquake_hazard_score"] >= 70:
        actions.append("Validate emergency inspection and shelter plans")
    if row["vulnerability_score"] >= 70:
        actions.append("Prioritize outreach for transport-constrained households")
    if row["response_readiness_index"] <= 40:
        actions.append("Increase responder and route readiness posture")
    if row["recovery_capacity_index"] <= 40:
        actions.append("Pre-stage utility restoration support")
    if not actions:
        actions.append("Maintain monitoring and readiness checks")
    return " | ".join(dict.fromkeys(actions))


priority = idx_df.sort_values("priority_index_conf_adj", ascending=False).copy()
priority["recommended_actions"] = priority.apply(recommend_actions, axis=1)
priority["rank"] = range(1, len(priority) + 1)

out_cols = [
    "rank", "municipio", "priority_band", "priority_index_conf_adj",
    "hazard_combined", "flood_hazard_muni", "earthquake_hazard_score",
    "vulnerability_score", "response_readiness_index", "recovery_capacity_index",
    "confidence_score", "recommended_actions"
]

priority_out = priority[out_cols].copy()
csv_out = OUTPUT_DIR / "priority_actions.csv"
priority_out.to_csv(csv_out, index=False)

# Lightweight HTML product
html_out = OUTPUT_DIR / "index_priority_overview.html"
html = []
html.append("<html><head><meta charset='utf-8'><title>GMU Priority Overview</title>")
html.append("<style>body{font-family:Arial,sans-serif;margin:20px;} table{border-collapse:collapse;width:100%;} th,td{border:1px solid #ddd;padding:8px;} th{background:#1f3a56;color:#fff;} .Red{background:#f8d7da;} .Orange{background:#fff3cd;} .Yellow{background:#fffbe6;} .Green{background:#e7f6e7;}</style>")
html.append("</head><body>")
html.append("<h1>Municipio Priority Overview</h1>")
html.append(f"<p>Rows: {len(priority_out)} | Generated UTC: {pd.Timestamp.utcnow()}</p>")

render_df = priority_out.copy()
render_df["priority_band_class"] = render_df["priority_band"]

# manual table for band row coloring
html.append("<table><thead><tr>")
for col in [c for c in render_df.columns if c != "priority_band_class"]:
    html.append(f"<th>{col}</th>")
html.append("</tr></thead><tbody>")

for _, r in render_df.iterrows():
    cls = r["priority_band_class"]
    html.append(f"<tr class='{cls}'>")
    for col in [c for c in render_df.columns if c != "priority_band_class"]:
        html.append(f"<td>{r[col]}</td>")
    html.append("</tr>")

html.append("</tbody></table></body></html>")
html_out.write_text("".join(html), encoding="utf-8")

print(f"Outputs:\n  {csv_out}\n  {html_out}")
display(priority_out.head(15))
